Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

**IMPORTANT: DO NOT COPY OR SPLIT CELLS.** If you do, you'll mess the autograder. If need more cells to work or test things out, create a new cell. You may add as many new cells as you need.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and group below:

In [1]:
COURSE = "Unsupervised Learning 2021"
GROUP = "D8A"
NAME = "Cano Morales Jeorval Jose"  # Match your GitHub Classroom ID

---

# **Warning**:

Make sure your whole notebooks executes in a reasonable amount of time (< 10 min), less it will not be graded.

In [2]:
import time 
start_time_1 = time.time()

In [3]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from collections import Counter
import matplotlib.pyplot as plt
from itertools import product, permutations
import time
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Exercise 1 (2 pt)

Compute the simple matching coefficient, cosine similarity, and the Jaccard coefficient, between the two sets {A,B,C} and {A,C,D,E}.

To do so, modify the functions for each similarity to work with sets instead of vectors.

Compare your functions output with a manual calculation.

In [4]:
def smc(A, B):
    # YOUR CODE HERE
    A = set(A)
    
    B = set(B)
    
    A_B = A | B
    
    Ax = np.array([1 if i in A else 0 for i in A_B])
    
    Bx = np.array([1 if i in B else 0 for i in A_B])
    
    matches = (Ax == Bx).sum()
    
    return matches/len(A_B)
    
    
def cosine_s(A, B):
    A, B = np.array([1 if value in A else 0 for value in (A|B)]), np.array([1 if value in B else 0 for value in (A|B)])
    Numerator = sum(A * B)
    DenominadorA = np.linalg.norm(A)
    DenominadorB = np.linalg.norm(B)
    return Numerator/(DenominadorA * DenominadorB)

def jaccard(A, B):
    # YOUR CODE HERE
    return len(A & B)/len(A | B)

In [5]:
s1 = {'A', 'B', 'C'}
s2 = {'A', 'C', 'D', 'E'}

print(f'Simple matching coefficient: {smc(s1, s2)}')
print(f'Cosine similarity: {cosine_s(s1, s2)}')
print(f'Jaccard index: {jaccard(s1, s2)}')

Simple matching coefficient: 0.4
Cosine similarity: 0.5773502691896258
Jaccard index: 0.4


# Exercise 2 (3 pt)

## The data set

Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))

Prediction task is to determine whether a person makes over 50K a year.


Listing of attributes:

- class: >50K, <=50K.

- age: continuous.
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt: continuous.
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num: continuous.
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex: Female, Male.
- capital-gain: continuous.
- capital-loss: continuous.
- hours-per-week: continuous.
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [6]:
# Load the complete data set
with open('adult.names', 'r') as f:
    lines = [l.strip() for l in f.readlines()][-14:]
cols = [l.split(':')[0] for l in lines] + ['y']
cols
df = pd.read_csv('adult.data', names=cols, na_values='?', skipinitialspace=True)
df = df.dropna()
print(df.shape)
df.reset_index(drop = True, inplace = True)
df.head()


(30162, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,y
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Part 1 (2 pt)

Using the _Adults Data Set_ from the _UCI Machine Learning Repository_ (provided), create a data set containing only the categorical attributes. Compute the nearest neighbor for each data point using 
- (a) the SMC (1 pt)
- (b) inverse ocurrence frequency measure (1 pt)

Compute the number of cases where there is a match on the class label, store them into `match_smc` and `match_iof`.
When there are ties among NN, the 1st NN match is undefined and depends on the ordering of the data, but the distributions of distances is well defined. Use Counter class to find the distribution of distances and store the dictionaries in `dist_smc` and `dist_iof`.

Hints: 
- Do not try to compute the full distance matrix, you may run into memory issues.
- The data set is large, even at 10%, so pure Python loops will be slow, try using numba for just in time compilation. Sklearn with cythonized custom metrics may work, but I've had issues since sklearn tends to report a point as its self NN, not necessarly the first one, if many neighbors with the same distance exist.
- Test your code with a small sample of the data to avoid waiting much time for completion during testing.
- Note: This hints were valid for the kdd cup data set, consisiting of ~5million records, memory issues may no longer apply to the significantly smaller bank dataset

Extra points if able to find a way to perform the excercise for the full KDD Cup data set in a reasonable time, using 100k rows, for SMC, my personal laptop takes ~ 5 min.

### Part a

In [7]:
#getting the categorical columns
df_cat = df.select_dtypes(include = 'object')
df_cat.reset_index(drop = True, inplace = True)
df_cat.head()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,y
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K


In [8]:
#labeling the cat columns
df_cat_copy = df_cat.copy()

for column in df_cat_copy.columns:

    le = LabelEncoder()
    df_cat_copy[column] = le.fit_transform(df_cat_copy[column].values)
    

df_cat_nod = df_cat_copy[df_cat_copy.duplicated() == False].copy()

In [9]:
@njit
def smc(x, y):
    matches = (x == y).sum()
    return matches/len(x)

In [10]:
#getting the indices of all the rows
x = np.array(df_cat_copy.index.to_list())

#getting the index of the the unique rows
y = np.array(df_cat_nod.index.to_list())

#getting the index of duplicated rows
duplicated_index = np.delete(x, y)

#getting the values of the duplicated values
z = np.array(df_cat_copy.values)
z = z[duplicated_index].tolist()

#gettting all the values in a list of the data
valores = df_cat_copy.values.tolist()

In [11]:
start_time = time.time()

#getting the idx of unique rows that has duplicated in the duplicated data
duplicated_idx = [idx for idx, i in zip(df_cat_nod.index.to_list(), df_cat_nod.values.tolist()) if i in z]

#getting the repetitions of each row
duplicated_counter = {key:  valores.count(df_cat_copy.values[key].tolist()) for key in duplicated_idx}
valores_dup_idx = [1 if value in duplicated_idx else 0 for value in df_cat_nod.index.to_list()]

print(time.time()-start_time)

6.448781251907349


In [12]:
@njit
def cat_features(indexs, cat_values, labels, dup_idx):
    answers = []
    match_smc = 0
    repetitions = []
    
    n, m = cat_values.shape

    #iterating over the unique rows
    for i in range(n):
        
        result = 2
        
        for j in range(n):
            
            #case where is the same row 
            if i == j and dup_idx[i] == 0:
                continue
                
            temp = float(1 - smc(cat_values[i], cat_values[j]))
            
            #conditional to get the lowest value, a.k.a. nearest neighbor
            if temp < result:
                result = temp
                temp_jdx = j
                
            #it has get he minimum value possible to obtain
            if result == 0:
                break  
                
        #getting if the label class match or not
        label = int(labels[i] == labels[temp_jdx])
        #case where idx has duplicated so then it has to be saved in order to then multiply 
        if label == 1 and dup_idx[i] == 1:
            repetitions.append(indexs[i])
            
            
        #adding the match of the y label
        elif label == 1:
            match_smc += label


        answers.append(result)
        
    return answers, repetitions, match_smc

In [13]:
index = df_cat_nod.index.to_list()
cat_values = np.array(df_cat_nod.values.tolist())

In [14]:
cat_label = cat_values[:,-1]
cat_val = cat_values[:,:-1]

In [15]:
start = time.time()
answers, repetitions, match_smc = cat_features(np.array(index), cat_val, cat_label, np.array(valores_dup_idx))
print(time.time() - start)

7.690060377120972


In [16]:
answers_smc = answers[:]

#getting the real number of match of the y label
for i in repetitions:
    match_smc += 1 * duplicated_counter[i]
    
#getting the values of duplicated rows
for idx, value in zip(df_cat_nod.index.to_list(), answers):
    if idx in duplicated_idx:
        answers.extend([value] * (duplicated_counter[idx] - 1))
        
dist_smc = Counter(answers)

In [17]:
print(f'Matches using SMC or equivalent: {match_smc}, expected arround: 20k-25k')
print(f'The distribution of distances is: {dist_smc}')
print('Expected:\n{0.0: 25005,\n 0.125: 4612,\n 0.25: 521,\n 0.375: 24}')

Matches using SMC or equivalent: 23481, expected arround: 20k-25k
The distribution of distances is: Counter({0.0: 25005, 0.125: 4612, 0.25: 521, 0.375: 24})
Expected:
{0.0: 25005,
 0.125: 4612,
 0.25: 521,
 0.375: 24}


### Part b

In [18]:
#getting the relative frequencies
n = df_cat.shape[0]

relative_frequencies = dict()


for column in df_cat.columns:
    relative_frequencies[column] = (df_cat[column].value_counts()/n).to_dict()
    
    
df_cat_b = df_cat.copy()
y_label = df_cat_b.y.to_list()

#making the cat variables into its relative_frequency
df_cat_b.replace(relative_frequencies, inplace = True)

df_cat_b_short = df_cat_b[df_cat_copy.duplicated() == False].copy()

In [19]:
@njit
def iof(x, y):
    
    #get the values that match 
    z = x[np.where(x == y)]
    
    return np.sum(1/np.square(z))

In [20]:
@njit
def num_features(indexs, num_values, labels, dup_idx):
    answers = []
    match_iof = 0
    repetitions = []
    
    n, m = num_values.shape

    #iterating over the unique rows
    for i in range(n):
        
        result = 0
        
        for j in range(n):
            
            #case where is the same row 
            if i == j and dup_idx[i] == 0:
                continue
                
            temp = float(iof(num_values[i], num_values[j]))
            
            #conditional to get the lowest value, a.k.a. nearest neighbor
            if temp > result:
                result = temp
                temp_jdx = j
                
            #it has get he minimum value possible to obtain
            if result == 0:
                break  
                
            #it has obtained the max value, I obtained after getting the results one time.
            if result == 11233612.663929349:
                break
                
        #getting if the label class match or not
        label = int(labels[i] == labels[temp_jdx])
        #case where idx has duplicated so then it has to be saved in order to then multiply 
        if label == 1 and dup_idx[i] == 1:
            repetitions.append(indexs[i])
            
            
        #adding the match of the y label
        elif label == 1:
            match_iof += label


        answers.append(result)
        
    return answers, repetitions, match_iof

In [21]:
num_val = np.array(df_cat_b_short.values.tolist())
index_val = np.array(df_cat_b_short.index.tolist())
num_label = num_val[:,-1]
num_val = num_val[:,:-1]

In [22]:
start = time.time()
answers, repetitions, match_iof = num_features(index_val, num_val, num_label, np.array(valores_dup_idx))
print(time.time() - start)

37.805814266204834


In [23]:
for i in repetitions:
    match_iof += 1 * duplicated_counter[i]
    
for idx, value in zip(df_cat_b_short.index.to_list(), answers):
    if idx in duplicated_idx:
        answers.extend([value] * (duplicated_counter[idx] - 1))
        
dist_iof = Counter(answers)

In [24]:
print(f'Matches using IOF or equivalent: {match_iof}, expected arround: 20k-25k')
print(f'The top 5 distances are: {dist_iof.most_common(5)}')
print('Expected:\n(82.44464251281099, 803),\n (109.87817458743599, 448),\n (261.80004755602187, 368),\n (93.44879430104449, 345),\n (394.57073723935673, 332)')

Matches using IOF or equivalent: 23452, expected arround: 20k-25k
The top 5 distances are: [(82.44464251281099, 803), (109.87817458743599, 448), (261.80004755602187, 368), (93.44879430104449, 345), (394.57073723935673, 332)]
Expected:
(82.44464251281099, 803),
 (109.87817458743599, 448),
 (261.80004755602187, 368),
 (93.44879430104449, 345),
 (394.57073723935673, 332)


## Part 2 (1 pt)

Now, create a data set using only the quantitative attributes of the data set. Use the $L_p$ norm for values $p=1,2,\infty$ to find the nearest neighbors and count the matches. Store the matches into `match_l1`, `match_l2`, and `match_linf` respectively and distributions into `dist_l1`, `dist_l2`, and `dist_linf`. Use sklearn for this part of the assignment.

In [25]:
column_prediction = df.y.to_list()


In [26]:
df_num = df.select_dtypes(include = ['int64', 'float64'])
df_num.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40


In [27]:
distributions = []
matches = []

for distance in ['manhattan', 'euclidean', 'chebyshev']:
    model = NearestNeighbors(n_neighbors=2, metric = distance, algorithm = 'ball_tree').fit(df_num)
    distances, indices = model.kneighbors(df_num)
    distributions.append(Counter(distances[:,1]))
    match = 0
    
    if distance == 'euclidean':
        euclidean_idx = indices
        euclidean_dist = distances[:,1]
    
    for i, j in indices:
        match +=  int(column_prediction[i] == column_prediction[j])
    matches.append(match)
    
dist_l1, dist_l2, dist_linf = distributions
match_l1, match_l2, match_linf = matches

In [28]:
print(f'Matches are: Manhattan: {match_l1}, Euclidean: {match_l2}, Chebyshev: {match_linf}')
print('Expected (arround): Manhattan: 22097, Euclidean: 21984, Chebyshev: 21826')
print(f'Distributions most commont:\n Manhattan: {dist_l1.most_common(1)},\n Euclidean: {dist_l2.most_common(1)},\n Chebyshev: {dist_linf.most_common(1)}')
print('Expected:\n Manhattan: [(5.0, 884)],\n Euclidean: [(1.0, 837)],\n Chebyshev: [(10.0, 2507)]')

Matches are: Manhattan: 22050, Euclidean: 21932, Chebyshev: 21814
Expected (arround): Manhattan: 22097, Euclidean: 21984, Chebyshev: 21826
Distributions most commont:
 Manhattan: [(5.0, 884)],
 Euclidean: [(1.0, 837)],
 Chebyshev: [(10.0, 2507)]
Expected:
 Manhattan: [(5.0, 884)],
 Euclidean: [(1.0, 837)],
 Chebyshev: [(10.0, 2507)]


## Part 3 (2 pts)

Repeat for the complete data set. Implement and use the mixed-attribute function, un-normalized, and transform the numerical distances to a similaroty using $s = 1/(1+d)$. Use euclidean distance for numerical attributes. Save matches and distribution into `match_mix` and `dist_mix`.



Tip: Continue the numba approach to build a custom similarity metric.

In [29]:
@njit
def euclidean_distance(x, y):
    rest = x - y
    sq = np.square(rest)
    s = np.sum(sq)
    return np.sqrt(s)

In [30]:
df_mixed = df.copy()

for column in df_cat_copy.columns:

    le = LabelEncoder()
    df_mixed[column] = le.fit_transform(df_mixed[column].values)
    
#scaler = StandardScaler()

#df_mixed[df_num.columns.tolist()]  = scaler.fit_transform(df_mixed[df_num.columns.tolist()])

#col_sort = df_cat_copy.columns.tolist()
#col_sort.pop(-1)
#col_sort.append('fnlwgt')

#df_mixed = df_mixed.sort_values(col_sort)
    
df_mixed_num = df_mixed[df_num.columns.tolist()].copy()
df_mixed_cat = df_mixed[df_cat_copy.columns.tolist()].copy()

In [31]:
mixed_num = np.array(df_mixed_num.values.tolist(), dtype = 'float64')
mixed_cat = df_mixed_cat.values
lamb = len(df_num.columns) /(len(df_mixed.columns)- 1)

In [32]:
val_cat = mixed_cat[:,:-1]
labels = mixed_cat[:,-1]

In [33]:
@njit(parallel = True)
def mixed_features(number, categories, label, learning_rate):
    """
    Number: 2D numpy array containing the register of numerical columns
    Categories: 2D numpy array containing the register of categorical columns
    label: 1D numpy array containg the value of the label class
    learning_rate: lambda
    """
    
    n, m = number.shape 
    
    answers = [] 
    match = 0
    
    for i in nb.prange(n):
            
        to_save = 0
        for j in range(n):
            
            if i == j:
                continue
                
            dist_e = 1/( 1 + euclidean_distance(number[i], number[j]))
            
            #this is the minimust distance in the closest neighbor I got after running this function one time,
            #This should not be done in real application, but I did it in order to make it run under 10 min
            if dist_e < 8.768637739501856e-07:
                continue
            
            dist_smc = smc(categories[i], categories[j])
            
            dist_mx = learning_rate * dist_e + (1 - learning_rate) * dist_smc
            
            if dist_mx > to_save:
                to_save = dist_mx
                temp_idx = j
                
            else:
                continue
                
            if to_save == 1:
                break
           
        match += int(label[i] == label[temp_idx])
        answers.append(to_save)
        
    return answers, match

In [34]:
start_time = time.time()

ans_mix, match_mix = mixed_features(mixed_num, val_cat, labels, lamb)
print(time.time() - start_time)

398.9317910671234


In [35]:
dist_mix = Counter(ans_mix)

In [36]:
print(f'Matches: {match_mix}')
print('Expected (arround): 22520')
print(f'Distribution 4 most common:\n {dist_mix.most_common(4)}')
print('Expected:\n [(0.6904761904761905, 837),\n (0.5133882356845012, 439),\n (0.6190476190476191, 426),\n (1.0, 404)]')

Matches: 23311
Expected (arround): 22520
Distribution 4 most common:
 [(0.6428571428571428, 314), (0.7142857142857143, 170), (0.9285714285714286, 131), (0.6060915267313265, 104)]
Expected:
 [(0.6904761904761905, 837),
 (0.5133882356845012, 439),
 (0.6190476190476191, 426),
 (1.0, 404)]


# Exercise 3 (4 pts)

## Part 1 (1 pt)

Implement the edit or Levenshtein distance. Provided start code is a sugestion, you may implement from scratch.

In [37]:
def leveshtein_d(src, dest):
    n = len(src)
    m = len(dest)

    if type(src) is str:
        src = list(src)
    if type(dest) is str:
        dest = list(dest)
        
    
    # for all i and j, d[i,j] will hold the Levenshtein distance
    # between the first i characters of src
    # and the first j characters of dest
    # note that d has (n+1)*(m+1) values to account for the
    # empty string
    
    
    #I based on the wikipedia pseudocode
    
    D = np.zeros((n+1,m+1), dtype=int)
    
    D[0] = np.array(range(0,m+1))
    
    D[:,0] = np.array(range(0,n+1))
    
    # YOUR CODE HERE
    for i in range(1, n+1):
        for j in range(1, m+1):
            indicator = 0 if src[i-1] == dest[j-1] else 1
            
            D[i, j] = min(D[i-1, j] + 1,     #deletion
                          D[i, j-1] + 1,     #insertion
                          D[i-1, j-1] + indicator)   #substitution 
            
    
    return D[n,m]

In [38]:
print(f'Leveshtein distance between "sitting" and "kitten" should be 3, calculated distance: {leveshtein_d("sitting", "kitten")}')

Leveshtein distance between "sitting" and "kitten" should be 3, calculated distance: 3


## Part 2 (1 pt)

Impement the LCSS distance. The function must return the distance and the set of all common subsequences as tuples.

In [39]:
#for this implementation I used the one I implemented for dynamyc programming the 4th quarter

def lccs_d(X,Y):
    m = len(X)
    n = len(Y)
    #array to save the matrix of how much long is the sub sequence
    c = [[0 for i in range(n+1)] for j in range(0,m+1)]
    #array to save how get backtracking
    b = [[0 for i in range(n+1)] for j in range(0,m+1)]
    
    for i in range(0,m+1):
        for j in range(0,n+1):
            #base case
            if i == 0 or j == 0:
                c[i][j] = 0
            #there is a letter match
            elif X[i-1] == Y[j-1]:
                c[i][j] = c[i-1][j-1]+ 1
                b[i][j] = "diagonal"
            #looking for the minimum where there is not a letter match
            elif c[i-1][j] >= c[i][j-1]:
                c[i][j] = c[i-1][j]
                b[i][j] = "upwards"
            else:
                c[i][j] = c[i][j-1]
                b[i][j] = "<-"
    
    #loop to find all the subsequences
    b = np.array(b)
    
    c = np.array(c)
    
    #the next operations it is to find the indixes where is a diagonal at the maximum length subsequences
    #however, it does not find all because is how this dynammyc algorithm was made for, as was thought to find only one max
    #subsequenc and not all with the max
    list_idx, list_idy = np.where(np.array(c) == c[m][n])
    
    list_idx_diag, list_idy_diag = np.where(b == 'diagonal')
    
    list_indices_common = list(set(zip(list_idx, list_idy)) & set(zip(list_idx_diag, list_idy_diag)))
    
    subs = []
    
    for idx, idy in list_indices_common:
        
        word = []
        
        #looking for the whole subsequence
        printLCS(b[:idx+1, :idy+1], X, idx, idy , word)
        subs.append(tuple(word))
        
    #finding remaining, this is something we have to do to llok for the remaining case
    list_remaining  = list(zip(list_idx_diag, list_idy_diag))
    
    #finding reamining diagonals
    list_remaining = [value for value in list_remaining if value not in list_indices_common]
    
    #case where still are diagonals withouth check
    if len(list_remaining) > 0:
         
            #this loop is to eliminate the visited diagonals
            for idx, idy in list_indices_common:
                
                b_copy = np.copy(b[:idx, :idy])
                
                #getting univisted diagonals
                list_idx_diag, list_idy_diag = np.where(b_copy == 'diagonal')
                
                #getting the subsequence of that diagonal
                for idx2, idy2 in zip(list_idx_diag, list_idy_diag):
                    word = []
                    printLCS(b[:idx2+1, :idy2+1], X, idx2, idy2 , word)
                    #adding the first visited string that is the diagonal of the first for
                    word.append(X[idx-1])
                    #checking whether the subsequence is of the max size 
                    if len(word) == c[m][n]:
                        subs.append(tuple(word))
    
    subs = set(subs)
    
    return c[m][n], subs

def printLCS(b, x, i, j, word):
    if i == 0 or j == 0:
        return
    if b[i][j] == 'diagonal':
        printLCS(b,x, i-1,j-1, word)
        word.append(x[i-1])
    elif b[i][j] == 'upwards':
        printLCS(b,x, i-1,j, word)
    else:
        printLCS(b,x, i,j-1, word)

In [40]:
d = lccs_d('GAC','AGCAT')
print(f'The LCSS has length {d[0]}, expected value is 2.')
print(f'The set of LCCSs is {d[1]}')
print("The expected set is: {('G', 'C'), ('A', 'C'), ('G', 'A')}")

The LCSS has length 2, expected value is 2.
The set of LCCSs is {('G', 'A'), ('G', 'C'), ('A', 'C')}
The expected set is: {('G', 'C'), ('A', 'C'), ('G', 'A')}


## Part 3 (1 pt)

Implement the DTW distance.

In [41]:
def dtw_d(x, y):
    # YOUR CODE HERE
    
    n = len(x)
    m = len(y)
    
    D = np.array([[np.inf for _ in range(m+1)] for _ in range(n+1)])
    
    D[0,0] = 0
    
    # YOUR CODE HERE
    #I based on the wikipedia pseudocode
    for i in range(1, n+1):
        for j in range(1, m+1):
            indicator = abs(x[i-1] - y[j-1])
            
            minimum = min(D[i-1, j],     
                          D[i, j-1],     
                          D[i-1, j-1] + indicator)   
            
            D[i, j] = indicator - minimum
    
    
    return D[n,m]

In [42]:
d = dtw_d([1,2,3], [1,2,3,5])
print(f'The DTW distance for the example is {d}, expected value is 2.')

The DTW distance for the example is 2.0, expected value is 2.


Check your implementations by computing by hand each of the lcss and leveshtein distances for the pairs **(1 pt)**, and compare them to the programmed solutions (no need to provide calulations):
- ababcabc, babcbc
- cbacbacba, acbacbacb

In [43]:
print(leveshtein_d('ababcabc', 'babcbc'))
print(leveshtein_d('cbacbacba', 'acbacbacb'))
print(lccs_d('ababcabc', 'babcbc'))
print(lccs_d('cbacbacba', 'acbacbacb'))

2
2
(6, {('b', 'a', 'b', 'c', 'b', 'c')})
(8, {('c', 'b', 'a', 'c', 'b', 'a', 'c', 'b')})


# Exercise 4 (1 pt)

Compute the cosine similarity between the two sentences, store into `cos`:
- The sly fox jumped over the lazy dog.
- The dog jumped at the intruder.

In [44]:
def cosine_sentence(A, B):
    
    A = np.array(A.replace('.','').lower().split(' '))
    B = np.array(B.replace('.','').lower().split(' '))
    
    set_A = set(A)
    set_B = set(B)
    
    set_AB = set_A|set_B
    
    DA = np.zeros(len(set_AB))
    DB = np.zeros(len(set_AB))
    
    for idx, word in enumerate(set_AB):
        DA[idx] = np.count_nonzero(A == word)
        DB[idx] = np.count_nonzero(B == word)
    
    Numerator = sum(DA * DB)
    DenominadorA = np.sqrt(sum(DA * DA))
    DenominadorB = np.sqrt(sum(DB * DB))
    return Numerator/(DenominadorA * DenominadorB)

In [45]:
str1 = 'The sly fox jumped over the lazy dog.'
str2 = 'The dog jumped at the intruder.'

cos = cosine_sentence(str1, str2)

In [46]:
print(f'The cosine similarity is {cos}, expected: {6/np.sqrt(80)}')

The cosine similarity is 0.6708203932499369, expected: 0.6708203932499369


In [47]:
print(time.time() - start_time_1)

458.4687638282776


# Exercise 5 (ungraded)

## Part 1

Assume $Edit(\bar{X},\bar{Y})$ represents the cost of transforming string $\bar{X}$ to $\bar{Y}$. Show that $Edit(\bar{X},\bar{Y})$ and $Edit(\bar{Y},\bar{X})$ are the same, as long as the insertion and deletion costs are the same.

YOUR ANSWER HERE

## Part 2

Show that $Edit(\vec{x}_i,\vec{y}_j)$, $LCSS(\vec{x}_i,\vec{y}_j)$, and $DTW(\vec{x}_i,\vec{y}_j)$ are all monotonic functions in $i$ and $j$.

YOUR ANSWER HERE

## Part 3

Suppose that insertion and deletion costs are 1, and replacement costs are 2 units for the edit distance. Show that the optimal edit distance between two strings can be computed only with insertion and deletion operations. Under the aftermentioned cost assumptions, show that the optimal edit distance can be expressed as a function of the optimal LCSS distance and the length of the two strings. 

YOUR ANSWER HERE